In [28]:

%matplotlib inline
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from scipy import stats
warnings.filterwarnings('ignore')
import os
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
import glob
from keras.models import Sequential
from keras.layers import LSTM, Dense, Flatten, Dropout, Conv2D  ,MaxPooling2D, Reshape
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
from keras.optimizers import Adam
import tensorflow as tf

### Preparing Data

In [3]:
respeck_filepaths = glob.glob("../Respeck/*")
df1 = pd.DataFrame()
for rfp in respeck_filepaths:
    files = glob.glob(f"{rfp}/*")
    
    for file in files:
        [main_act,sub_act] = file.split(".csv")[0].split('_')[-2:]
        # main_activity = file.split(".csv")[0].split('_')[-2]
        
        df = pd.read_csv(file,index_col=0)
        df['activity'] = main_act
        df['sub_activity'] = sub_act
        df['user'] = rfp.split('\\')[-1]
        # print(df)
        df1 = df1.concat(df)

df1 = df1[df1['sub_activity'] == 'breathingNormal']     
df1.loc[df1['activity'].isin(('sitting', 'standing')),'activity'] = 'sitting_standing'

AttributeError: 'DataFrame' object has no attribute 'concat'

In [ ]:
columns = ['user','activity','timestamp', 'accel_x', 'accel_y', 'accel_z']
# df1 = df1[columns]
df_har = df1[columns]
# removing null values
df_har = df_har.dropna()
df_har.shape
# transforming the user to float
df_har['user'] = df_har['user'].str.replace('s', '')
df_har['user'] = df_har['user'].apply(lambda x:int(x))


In [ ]:
df_har.to_csv('task1.csv',index=False)

In [2]:
# ONLY RUN THIS AFTER CSV GENERATION
general_act_df = pd.read_csv('task1.csv')
general_act_df

,user,activity,timestamp,accel_x,accel_y,accel_z
0,1,ascending,1697605965,0.011963,-0.855774,-0.029846
1,1,ascending,1697606005,-0.001709,-0.826233,-0.036194
2,1,ascending,1697606045,-0.058838,-0.933899,-0.032532
3,1,ascending,1697606085,-0.002441,-1.115051,-0.028870
4,1,ascending,1697606125,-0.036621,-1.035217,-0.076477
...,...,...,...,...,...,...
639844,98,sitting_standing,1697636165,-0.057617,-0.985901,0.050232
639845,98,sitting_standing,1697636205,-0.061523,-0.980042,0.061951
639846,98,sitting_standing,1697636245,-0.067627,-1.000793,0.043396
639847,98,sitting_standing,1697636285,-0.057617,-0.976379,0.068054


### Segment Generation

In [3]:
random_seed = 42   
n_time_steps = 50 
n_features = 3 
step = 10
n_epochs = 20      
batch_size = 32

In [5]:
class CustomLabelEncoder:
    def _init_(self):
        self.class_mapping = {
            'sitting_standing': 0,
            'lyingLeft': 1,
            'lyingRight': 2,
            'lyingBack': 3,
            'lyingStomach': 4,
            'normalWalking': 5,
            'running': 6,
            'descending': 7,
            'ascending': 8,
            'shuffleWalking': 9,
            'miscMovement':10
        }

    def fit_transform(self, y):
        return [self.class_mapping[cls] for cls in y]

    def inverse_transform(self, y):
        reverse_mapping = {v: k for k, v in self.class_mapping.items()}
        return [reverse_mapping[val] for val in y]

In [58]:
def segments_overlap(data):
    segments = []
    labels = []

    for i in range(0,  data.shape[0]- n_time_steps, step):  

        xs = data['accel_x'].values[i: i + n_time_steps]

        ys = data['accel_y'].values[i: i + n_time_steps]

        zs = data['accel_z'].values[i: i + n_time_steps]

        label = stats.mode(data['activity'][i: i + n_time_steps])[0][0]

        segments.append([xs, ys, zs])

        labels.append(label)
        
    reshaped_segments = np.asarray(segments, dtype= np.float32).reshape(-1, n_time_steps, n_features)
    labels = np.asarray(labels).reshape(-1,1)

    # le = LabelEncoder()
    # labels = le.fit_transform(labels)
    enc = OneHotEncoder(handle_unknown='ignore').fit(labels)
    labels = enc.transform(labels).toarray()
    # labels = np.asarray(pd.get_dummies(labels), dtype = np.float32)
    # print(enc.categories_)
    return reshaped_segments,labels,enc.categories_

In [59]:
def segments_no_overlap(data):
    segments= []
    labels = []
    
    activities = data['activity'].unique()
    for act in activities:
        act_data = data[data['activity'] == act]
        
        # for i in range(0, len(all_data) - n_time_steps, step):
        for i in range(0,  act_data.shape[0]- n_time_steps, step):  

            xs = act_data['accel_x'].values[i: i + n_time_steps]
            ys = act_data['accel_y'].values[i: i + n_time_steps]
            zs = act_data['accel_z'].values[i: i + n_time_steps]

            segments.append([xs, ys, zs])
            labels.append(act)

    #reshape the segments which is (list of arrays) to a list
    reshaped_segments = np.asarray(segments, dtype= np.float32).reshape(-1, n_time_steps, n_features)

    labels = np.asarray(pd.get_dummies(labels), dtype = np.float32)
    
    return reshaped_segments,labels

### Model Training

In [60]:
from sklearn.model_selection import train_test_split

def test_train_split(seg,labls):    
    X_train, X_test, y_train, y_test = train_test_split(seg, labls, test_size = 0.2, random_state = random_seed)
    return X_train, X_test, y_train, y_test

In [61]:
def model_LSTM(X_train,y_train):
    model = Sequential()
    # RNN layer
    model.add(LSTM(units = 128, input_shape = (X_train.shape[1], X_train.shape[2])))
    # Dropout layer
    model.add(Dropout(0.5)) 
    # Dense layer with ReLu
    model.add(Dense(units = 64, activation='relu'))
    # Softmax layer
    model.add(Dense(y_train.shape[1], activation = 'softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    model.summary()
    return model 

In [63]:
def model_cnn(trainX, trainy):
    n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
    model = Sequential()
    model.add(Reshape((n_timesteps, n_features, 1)))
    model.add(Conv2D(filters=64, kernel_size=(3,1), activation='relu'))
    model.add(Conv2D(filters=64, kernel_size=(3,1), activation='relu'))
    model.add(Dropout(0.5))
    model.add(MaxPooling2D(pool_size=(2,1)))
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=n_epochs, batch_size=batch_size, verbose=1)
    # evaluate model
    return model

### Driver Code

In [64]:
accuracies = []
for user in general_act_df['user'].unique():
    
    train_df = general_act_df[general_act_df['user'] != user]
    test_df = general_act_df[general_act_df['user'] == user]

    X_train, y_train, categories = segments_overlap(train_df)
    X_test, y_test, cat2 = segments_overlap(test_df)
    
    # model = model_LSTM(X_train,y_train)
    # history = model.fit(X_train, y_train, epochs = n_epochs, validation_split = 0.20, batch_size = batch_size, verbose = 1)
    
    model  = model_cnn(X_train,y_train)
    
    loss, accuracy = model.evaluate(X_test, y_test, batch_size = batch_size, verbose = 1)
    print(f"Test Accuracy ({user}):", accuracy)
    print(f"Test Loss ({user}):", loss)
    
    # accuracies.append((user,loss,accuracy))
    break
    

Epoch 1/20
1971/1971 [==============================] - 27s 13ms/step - loss: 0.4678 - accuracy: 0.8346
Epoch 2/20
1971/1971 [==============================] - 26s 13ms/step - loss: 0.2695 - accuracy: 0.9061
Epoch 3/20
1971/1971 [==============================] - 26s 13ms/step - loss: 0.2080 - accuracy: 0.9276
Epoch 4/20
1971/1971 [==============================] - 29s 15ms/step - loss: 0.1727 - accuracy: 0.9409
Epoch 5/20
1971/1971 [==============================] - 29s 15ms/step - loss: 0.1475 - accuracy: 0.9491
Epoch 6/20
1971/1971 [==============================] - 30s 15ms/step - loss: 0.1267 - accuracy: 0.9563
Epoch 7/20
1971/1971 [==============================] - 29s 15ms/step - loss: 0.1153 - accuracy: 0.9609
Epoch 8/20
1971/1971 [==============================] - 30s 15ms/step - loss: 0.1031 - accuracy: 0.9646
Epoch 9/20
1971/1971 [==============================] - 30s 15ms/step - loss: 0.0931 - accuracy: 0.9667
Epoch 10/20
1971/1971 [==============================] - 29s 15m

In [100]:
categories

[array(['ascending', 'descending', 'lyingBack', 'lyingLeft', 'lyingRight',
        'lyingStomach', 'miscMovement', 'normalWalking', 'running',
        'shuffleWalking', 'sitting_standing'], dtype='<U16')]

In [90]:
import tensorflow_model_optimization as tfmot

In [91]:
quantize_model = tfmot.quantization.keras.quantize_model

q_aware_model = quantize_model(model)

# `quantize_model` requires a recompile.
q_aware_model.compile(optimizer='sgd',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

q_aware_model.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 quantize_layer_4 (QuantizeL  (None, 50, 3)            3         
 ayer)                                                           
                                                                 
 quant_reshape_7 (QuantizeWr  (None, 50, 3, 1)         1         
 apperV2)                                                        
                                                                 
 quant_conv2d_14 (QuantizeWr  (None, 48, 3, 64)        387       
 apperV2)                                                        
                                                                 
 quant_conv2d_15 (QuantizeWr  (None, 46, 3, 64)        12483     
 apperV2)                                                        
                                                                 
 quant_dropout_11 (QuantizeW  (None, 46, 3, 64)      

In [95]:
converter = tf.lite.TFLiteConverter.from_keras_model(q_aware_model)
converter.target_spec.supported_types = [tf.float16]
quantized_tflite_model = converter.convert()

INFO:tensorflow:Assets written to: C:\Users\chhal\AppData\Local\Temp\tmpruc2c2yz\assets


INFO:tensorflow:Assets written to: C:\Users\chhal\AppData\Local\Temp\tmpruc2c2yz\assets


In [101]:
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open(f'../models/cnn_model_task{1}_{n_time_steps}_{n_features}.tflite', 'wb') as f:
  f.write(quantized_tflite_model)

INFO:tensorflow:Assets written to: C:\Users\chhal\AppData\Local\Temp\tmp7vr0w4hs\assets


INFO:tensorflow:Assets written to: C:\Users\chhal\AppData\Local\Temp\tmp7vr0w4hs\assets


In [98]:
interpreter = tf.lite.Interpreter(model_path="../models/cnn_model_quant1_task1_50_3.tflite")
interpreter.allocate_tensors()

# Get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test the model on one instance
interpreter.set_tensor(input_details[0]['index'], np.float32(X_test[0:1]))
interpreter.invoke()
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)

[[0.00304507 0.00304507 0.00710347 0.00780451 0.00304507 0.05904093
  0.00304507 0.00304507 0.00304507 0.00304507 0.90473557]]


In [99]:
correct_predictions = 0
total_predictions = 0

for i in range(len(X_test)):
    interpreter.set_tensor(input_details[0]['index'], np.float32(X_test[i:i+1]))
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    total_predictions += 1
    if np.argmax(output_data) == np.argmax(y_test[i]):
        correct_predictions += 1

accuracy = correct_predictions / total_predictions
print('Test accuracy:', accuracy)

Test accuracy: 0.16427783902976847
